<a href="https://colab.research.google.com/github/mojo46/Machine-Learning-noob/blob/master/Aphabets_Emnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data from kaggle 

Login to kaggle generate your api key AKA kaggle.json

In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

In [3]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 68 Nov 18 12:16 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d crawford/emnist

100% 1.24G/1.24G [00:27<00:00, 50.8MB/s]
100% 1.24G/1.24G [00:27<00:00, 48.2MB/s]


In [0]:
!unzip '/content/emnist.zip'

# Data prep

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils

Using TensorFlow backend.


Read csv into a dataframe

In [0]:
emnist_letters_train = pd.read_csv('/content/emnist-letters-train.csv', header=None)
emnist_letters_test = pd.read_csv('/content/emnist-letters-test.csv', header=None)

Convert dataframe to np array and one hot encode the lables

In [0]:
def prepare_data(x:pd):
  nparr = np.array(x)
  data = (nparr[:,1 :])/255
  lab = nparr[:,0]
  label = np_utils.to_categorical(lab,27,dtype='float32')
  return (data , label)

In [0]:
(emnist_letters_train_data1 ,emnist_letters_train_lable1 )=prepare_data(emnist_letters_train)
(emnist_letters_test_data1 ,emnist_letters_test_lable1 )=prepare_data(emnist_letters_test)

In [13]:
print(emnist_letters_train_data1[0],emnist_letters_train_lable1[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.00392157 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.07843137 0.44705882 0.49019608 0.32156863
 0.14509804 0.14509804 0.15294118 0.45098039 0.61568627 0.36862745
 0.08235294 0.03921569 0.1254902  0.1450

Simple number to alphabets function

In [0]:
def alphabet(x : int):
  if(x == 1): return 'A'
  elif(x==2): return 'B'
  elif(x==3): return 'C'
  elif(x==4): return 'D'
  elif(x==5): return 'E'
  elif(x==6): return 'F'
  elif(x==7): return 'G'
  elif(x==8): return 'H'
  elif(x==9): return 'I'
  elif(x==10): return 'J'
  elif(x==11): return 'K'
  elif(x==12): return 'L'
  elif(x==13): return 'M'
  elif(x==14): return 'N'
  elif(x==15): return 'O'
  elif(x==16): return 'P'
  elif(x==17): return 'Q'
  elif(x==18): return 'R'
  elif(x==19): return 'S'
  elif(x==20): return 'T'
  elif(x==21): return 'U'
  elif(x==22): return 'V'
  elif(x==23): return 'W'
  elif(x==24): return 'X'
  elif(x==25): return 'Y'
  elif(x==26): return 'Z'
  else :  return 'nothing'

Test images with random index

In [0]:
index = 10654
plt.imshow(emnist_letters_test_data1[index].reshape(28,28))
print(alphabet(emnist_letters_test_lable1[index]))

In [0]:
!cat /content/emnist-letters-mapping.txt

In [0]:
print(alphabet(1))

In [21]:
print(emnist_letters_train_data1.shape,emnist_letters_train_lable1[0])

(88800, 784) [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0.]


# Model Archi and Complie

In [0]:
model = Sequential()

model.add(Dense(512, input_dim=784))
# An "activation" is just a non-linear function applied to the output
# of the layer above. Here, with a "rectified linear unit",
# we clamp all values below 0 to 0.
model.add(Activation('relu'))
# Dropout helps protect the model from memorizing or "overfitting" the training data
#model.add(Dropout(0.2))

model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.2))

model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.2))

model.add(Dense(27))
# This special "softmax" activation among other things,
# ensures the output is a valid probaility distribution, that is
# that its values are all non-negative and sum to 1.
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model

In [0]:
hist = model.fit(emnist_letters_train_data1, emnist_letters_train_lable1, batch_size=None, nb_epoch=100, validation_data=(emnist_letters_test_data1, emnist_letters_test_lable1))

88800/88800 [==============================] - 22s 249us/step - loss: 0.5818 - acc: 0.8157 - val_loss: 0.4028 - val_acc: 0.8664
Epoch 2/100
88800/88800 [==============================] - 17s 193us/step - loss: 0.3302 - acc: 0.8915 - val_loss: 0.3706 - val_acc: 0.8802
Epoch 3/100
88800/88800 [==============================] - 17s 195us/step - loss: 0.2674 - acc: 0.9101 - val_loss: 0.3908 - val_acc: 0.8776
Epoch 4/100
88800/88800 [==============================] - 17s 195us/step - loss: 0.2314 - acc: 0.9205 - val_loss: 0.3658 - val_acc: 0.8857
Epoch 5/100
88800/88800 [==============================] - 17s 194us/step - loss: 0.2057 - acc: 0.9270 - val_loss: 0.3693 - val_acc: 0.8911
Epoch 6/100
88800/88800 [==============================] - 17s 196us/step - loss: 0.1845 - acc: 0.9334 - val_loss: 0.3706 - val_acc: 0.8908
Epoch 7/100
88800/88800 [==============================] - 17s 192us/step - loss: 0.1734 - acc: 0.9368 - val_loss: 0.4062 - val_acc: 0.8880
Epoch 8/100
88800/88800 [=======

# Display results

In [0]:
plt.figure(figsize=(14,3))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(hist.history['loss'], color='b', label='Training Loss')
plt.plot(hist.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(hist.history['acc'], color='b', label='Training Accuracy')
plt.plot(hist.history['val_acc'], color='r', label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()